In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

movies_df['genres'] = movies_df['genres'].str.replace('|', ' ')

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['genres'])

content_cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

pivot_table = ratings_df.pivot_table(index='movieId', columns='userId', values='rating').fillna(0)
pivot_table_centered = pivot_table.apply(lambda x: x - x.mean(), axis=1)

collab_cosine_sim = linear_kernel(pivot_table_centered, pivot_table_centered)

def get_content_based_recommendations(title, cosine_sim=content_cosine_sim):
    idx = movies_df.loc[movies_df['title'].str.contains(title, case=False)].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Get top 10 similar movies
    movie_indices = [i[0] for i in sim_scores]
    return movies_df['title'].iloc[movie_indices]

def get_item_based_recommendations(movie_id, cosine_sim=collab_cosine_sim):
    idx = movies_df[movies_df['movieId'] == movie_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Get top 10 similar movies
    movie_indices = [i[0] for i in sim_scores]
    return movies_df['title'].iloc[movie_indices]

content_based_results = get_content_based_recommendations('Toy Story')
print("Content-Based Filtering Recommendations for Toy Story:")
print(content_based_results)

item_based_results = get_item_based_recommendations(1)  # Assuming Toy Story has movieId 1
print("\nItem-Based Collaborative Filtering Recommendations for Toy Story:")
print(item_based_results)

Content-Based Filtering Recommendations for Toy Story:
1706                                          Antz (1998)
2355                                   Toy Story 2 (1999)
2809       Adventures of Rocky and Bullwinkle, The (2000)
3000                     Emperor's New Groove, The (2000)
3568                                Monsters, Inc. (2001)
6194                                     Wild, The (2006)
6486                               Shrek the Third (2007)
6948                       Tale of Despereaux, The (2008)
7760    Asterix and the Vikings (Astérix et les Viking...
8219                                         Turbo (2013)
Name: title, dtype: object

Item-Based Collaborative Filtering Recommendations for Toy Story:
2353                                 'night Mother (1986)
224             Star Wars: Episode IV - A New Hope (1977)
910     Once Upon a Time in the West (C'era una volta ...
322                                 Lion King, The (1994)
418                                  Ju